Where we convert the raw data into training and validation sets.
Manila Observatory will be the "test" set.

`aerosol_type` is the response variable

In [1]:
import pandas as pd
import numpy as np
import os

# MICE (imputation)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model

# Feature Selection (VIF)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

c:\Users\Hans\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
REF_CLUSTER_DIR = "raw_data/reference_sites/"

site2type = {
    "Solar_Village" : "MD",
    "Beijing" : "PD",
    "Mongu" : "BBD",
    "Alta_Floresta": "BBW",
    "GSFC" : "UI",
    "Chen-Kung_Univ" : "UID",
}

site_min_month = {
    "Solar_Village" : 3,
    "Beijing" : 1,
    "Mongu" : 8,
    "Alta_Floresta": 8,
    "GSFC" : 6,
    "Chen-Kung_Univ" : 1,
}

site_max_month = {
    "Solar_Village" : 7,
    "Beijing" : 12,
    "Mongu" : 11,
    "Alta_Floresta": 10,
    "GSFC" : 9,
    "Chen-Kung_Univ" : 12,
}



cols_to_keep = [
    'Site',
    'Date(dd:mm:yyyy)',
    'Angstrom_Exponent_440-870nm_from_Coincident_Input_AOD',
    'Extinction_Angstrom_Exponent_440-870nm-Total',
    'Single_Scattering_Albedo[440nm]',
    'Single_Scattering_Albedo[675nm]',
    'Single_Scattering_Albedo[870nm]',
    'Single_Scattering_Albedo[1020nm]',
    'Absorption_Angstrom_Exponent_440-870nm',
    'Refractive_Index-Real_Part[440nm]',
    'Refractive_Index-Real_Part[675nm]',
    'Refractive_Index-Real_Part[870nm]',
    'Refractive_Index-Real_Part[1020nm]',
    'Refractive_Index-Imaginary_Part[440nm]',
    'Refractive_Index-Imaginary_Part[675nm]',
    'Refractive_Index-Imaginary_Part[870nm]',
    'Refractive_Index-Imaginary_Part[1020nm]',
    'Asymmetry_Factor-Total[440nm]',
    'Asymmetry_Factor-Total[675nm]',
    'Asymmetry_Factor-Total[870nm]',
    'Asymmetry_Factor-Total[1020nm]',
    'Asymmetry_Factor-Fine[440nm]',
    'Asymmetry_Factor-Fine[675nm]',
    'Asymmetry_Factor-Fine[870nm]',
    'Asymmetry_Factor-Fine[1020nm]',
    'Asymmetry_Factor-Coarse[440nm]',
    'Asymmetry_Factor-Coarse[675nm]',
    'Asymmetry_Factor-Coarse[870nm]',
    'Asymmetry_Factor-Coarse[1020nm]',
    'Sphericity_Factor(%)',
    'Lidar_Ratio[440nm]',
    'Lidar_Ratio[675nm]',
    'Lidar_Ratio[870nm]',
    'Lidar_Ratio[1020nm]',
    'Depolarization_Ratio[440nm]',
    'Depolarization_Ratio[675nm]',
    'Depolarization_Ratio[870nm]',
    'Depolarization_Ratio[1020nm]',
]

In [3]:
df = pd.concat([
    pd.read_csv(
        REF_CLUSTER_DIR  + f, skiprows=6, na_values=-999
    )[cols_to_keep].assign(
        date = lambda x: pd.to_datetime(x["Date(dd:mm:yyyy)"], format=r"%d:%m:%Y"),
        month = lambda x: x.date.dt.month,
        aerosol_type = lambda x: x.Site.map(site2type),
        start_month = lambda x: x.Site.map(site_min_month),
        end_month = lambda x: x.Site.map(site_max_month),
    ).query("start_month <= month <= end_month")\
    .drop(columns=["Site", "Date(dd:mm:yyyy)", "month", "start_month", "end_month"])
    for f in os.listdir(REF_CLUSTER_DIR)
])

In [4]:
df.aerosol_type.value_counts()

MD     6162
UI     3093
PD     3088
BBD    1984
UID    1550
BBW    1060
Name: aerosol_type, dtype: int64

In [5]:
df["Single_Scattering_Albedo[440nm]"].isna().mean()

0.4823758634941253

# Train-Validation Split

Data must be split before doing imputation in order to avoid data leakage. To keep things simple, we'll stick with a simple 70-30 split.

In [6]:
np.random.seed(2023)
rand_idx = np.random.permutation(df.index)
train_idx = rand_idx[:int(0.7*len(rand_idx))]
valid_idx = rand_idx[int(0.7*len(rand_idx)):]

In [7]:
train_df = df.loc[train_idx]
valid_df = df.loc[valid_idx]

# MICE imputation

SMOTE is for balancing the classes, in our case what we need is a method for imputation. A common method to use (apart from filling the NANs with the mean) is Multivariate Imputation by Chained Equation (MICE).

Ref: https://towardsdatascience.com/imputing-missing-data-with-simple-and-advanced-techniques-f5c7b157fb87#:~:text=One%20way%20to%20impute%20missing,with%20the%20previously%20observed%20value.

We fit the MICE imputer on the training set and use the learned model for imputing the validation and test sets.

In [8]:
train_df.columns[train_df.isna().mean()==1]

Index(['Sphericity_Factor(%)'], dtype='object')

In [9]:
# remove Sphericity_Factor(%) because the entire column is null.
X_cols = [c for c in train_df.columns if c not in ["aerosol_type", "date", "Sphericity_Factor(%)"]]

In [10]:
mice_imputer = IterativeImputer(
    estimator=linear_model.BayesianRidge(), 
    n_nearest_features=None,
    imputation_order="ascending"    
).fit(
    train_df[X_cols]
)

c:\Users\Hans\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [11]:
train_df = pd.concat([
    train_df.reset_index()[["aerosol_type", "date"]],
    pd.DataFrame(
        mice_imputer.transform(train_df[X_cols]),
        columns=X_cols
    )
], axis=1)

# Normalize Data

In [12]:
train_mean = train_df[X_cols].mean()
train_std = train_df[X_cols].std()

In [13]:
train_df[X_cols] = (train_df[X_cols] - train_mean)/ train_std

# Feature Selection

A lot of the columns are highly correlated -- let's multicollinearity using Variance Inflation Factor (VIF).

In [14]:
vif_values = {}
x = train_df[X_cols].values
for i, col in enumerate(X_cols):
    vif = variance_inflation_factor(x, i)
    vif_values[col] = vif

In [15]:
sorted_vif = pd.DataFrame(vif_values.values(), index=vif_values.keys()).sort_values(0, ascending=True).reset_index()
sorted_vif

,index,0
0,Absorption_Angstrom_Exponent_440-870nm,12.614950
1,Lidar_Ratio[440nm],45.293721
2,Refractive_Index-Imaginary_Part[440nm],113.871125
3,Asymmetry_Factor-Fine[440nm],116.673570
4,Refractive_Index-Real_Part[440nm],151.905500
5,Single_Scattering_Albedo[440nm],167.986787
6,Asymmetry_Factor-Total[440nm],230.307581
7,Asymmetry_Factor-Coarse[440nm],240.067637
8,Depolarization_Ratio[440nm],288.913369
9,Refractive_Index-Real_Part[1020nm],370.600901


In [16]:
vif_values = {}
cols = sorted_vif.iloc[:10, 0]
x = train_df[cols].values
for i, col in enumerate(cols):
    vif = variance_inflation_factor(x, i)
    vif_values[col] = vif

In [17]:
sorted_vif = pd.DataFrame(vif_values.values(), index=vif_values.keys()).sort_values(0, ascending=True).reset_index()
sorted_vif

,index,0
0,Absorption_Angstrom_Exponent_440-870nm,1.916303
1,Asymmetry_Factor-Coarse[440nm],2.213302
2,Lidar_Ratio[440nm],3.542581
3,Depolarization_Ratio[440nm],3.809359
4,Refractive_Index-Real_Part[1020nm],4.166197
5,Refractive_Index-Real_Part[440nm],5.577222
6,Asymmetry_Factor-Total[440nm],6.624543
7,Asymmetry_Factor-Fine[440nm],6.710713
8,Refractive_Index-Imaginary_Part[440nm],12.464324
9,Single_Scattering_Albedo[440nm],13.784023


By trial-and-error, we only need 10 features in order to remove the multicollinearlity. Although the usual filter for VIF is 5, we have to be less strict in this case since we know from domain knowledge that some of the high VIF features are still relevant. For example, Single Scattering Albedo is known an important indicator of reflectivity, so we still include it despite having a VIF > 10.

In [18]:
cols_to_use = ["aerosol_type", "date"] + list(sorted_vif.iloc[:,0])

# Finalize Train and Validation Sets

In [19]:
train_df[cols_to_use].to_csv("train_set.csv", index=False)

In [20]:
valid_temp = pd.DataFrame(mice_imputer.transform(valid_df[X_cols]), columns=X_cols)
valid_temp = (valid_temp - train_mean)/train_std

In [21]:
pd.concat([
    valid_df[["aerosol_type", "date"]].reset_index(),
    valid_temp
], axis=1)[cols_to_use].to_csv("valid_set.csv", index=False)

# Test Set (Manila Observatory)

In [22]:
test_df = pd.read_csv(
    "./raw_data/20090101_20221231_Manila_Observatory.all", skiprows=6, na_values=-999
)[cols_to_keep].assign(
    date = lambda x: pd.to_datetime(x["Date(dd:mm:yyyy)"], format=r"%d:%m:%Y"),
).drop(columns=["Site", "Date(dd:mm:yyyy)"])

In [23]:
test_temp = pd.DataFrame(mice_imputer.transform(test_df[X_cols]), columns=X_cols)
test_temp = (test_temp - train_mean)/ train_std

In [24]:
test_cols_to_use = [c for c in cols_to_use if c != "aerosol_type"]

In [25]:
test_temp.assign(
    date = test_df.date.values
)[test_cols_to_use].to_csv("test_set.csv", index=False)